In [72]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
from collections import Counter
from scipy.interpolate import interp1d

In [3]:
def seismic_ascii_parser(file_path):
    all_data = []
    head = ""
    xarray = []
    yarray = []
    zarray = []
    with open(file_path, 'r') as f:
        content = f.readlines()
        for line in content:
            if '#' in line:
                head += line
                continue
            l = line.split(' ')
            xarray.append(float(l[0]))
            yarray.append(float(l[1]))
            zarray.append(float(l[2]))

    df = pd.DataFrame({'x': xarray, 'y': yarray, 'z': zarray})
    all_data.append({"test": df})
    return create_seismic_ascii_date(all_data)[0]

def create_seismic_ascii_date(data_frame):
    for dataframe_data in data_frame:
        _, value = next(iter(dataframe_data.items()))
        return value.to_dict("records"),


In [4]:
# Это списки со словарями формата [{'x': 445946.632991, 'y': 7004198.214381, 'z': -2884.616455}, ...]
Lateral_trend = seismic_ascii_parser(r"C:/HV/Seismic/datas/test_3_5/SFA_4_klass.txt")
Top = seismic_ascii_parser(r"C:/HV/Seismic/datas/test_3_5/TVDSS_Top_U1.txt")
Bottom = seismic_ascii_parser(r"C:/HV/Seismic/datas/test_3_5/TVDSS_Bottom_U1.txt")

In [5]:
df_Lateral_trend = pd.DataFrame(Lateral_trend)
df_Top_U1 = pd.DataFrame(Top)
df_Bottom_U1 = pd.DataFrame(Bottom)

In [6]:
#Датафрейм из имен скважин и их координатами. 
column_names = ['wells', 'name', 'inlines', 'xlines', 'top', 'bottom'] 
df_all_wells_coor = pd.read_csv('C:/HV/Seismic/datas/test_3_5/data_wells.txt', sep='\t', names=column_names, header=None)

In [7]:
df_Lateral_trend = df_Lateral_trend.sort_values('x')
df_Top_U1 = df_Top_U1.sort_values('x')
df_Bottom_U1 = df_Bottom_U1.sort_values('x')
df_all_wells_coor = df_all_wells_coor.sort_values('inlines')

merged_lateral = pd.merge_asof(df_all_wells_coor, df_Lateral_trend, left_on='inlines', right_on='x', direction='nearest')
merged_lateral = merged_lateral.sort_values('xlines')

df_Top_U1 = df_Top_U1.sort_values('y')
merged_top = pd.merge_asof(merged_lateral, df_Top_U1, left_on='xlines', right_on='y', direction='nearest', suffixes=('', '_top'))
merged_top = merged_top.sort_values('xlines')

df_Bottom_U1 = df_Bottom_U1.sort_values('y')
merged_bottom = pd.merge_asof(merged_top, df_Bottom_U1, left_on='xlines', right_on='y', direction='nearest', suffixes=('', '_bottom'))


result_df = merged_bottom[['wells', 'inlines', 'xlines', 'z', 'z_top', 'z_bottom']]
result_df.columns = ['wells', 'inlines', 'xlines', 'Lateral_trend', 'Top_TVD', 'Bottom_TVD']

In [8]:
#добавляем колонки по МД и переводим в словарь для упрощения
result_df.loc[:, 'Top_MD'] = None
result_df.loc[:, 'Bottom_MD'] = None
list_of_dicts = result_df.to_dict(orient='list')

C:\Users\User\AppData\Local\Temp\ipykernel_11444\3533139330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.loc[:, 'Top_MD'] = None
C:\Users\User\AppData\Local\Temp\ipykernel_11444\3533139330.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.loc[:, 'Bottom_MD'] = None


In [9]:
def process_well_data_3(folder_path, dict):
    copy_dict = dict.copy()
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            name_part = os.path.splitext(filename)[0]

            index = copy_dict['wells'].index(name_part)

            
            file_path = os.path.join(folder_path, filename)
            data = pd.read_csv(file_path, sep=r'\s+', comment='#', header=0, on_bad_lines='skip')

            if data.shape[1] >= 4:
                
                first_col_numeric = pd.to_numeric(data.iloc[:, 0], errors='coerce')
                first_col_interpolated = first_col_numeric.interpolate()
                fourth_col = data.iloc[:, 3]
               
                top_tvd = copy_dict['Top_TVD'][index]
                bottom_tvd = copy_dict['Bottom_TVD'][index]

                top_md = first_col_interpolated[fourth_col == top_tvd].values
                bottom_md = first_col_interpolated[fourth_col == bottom_tvd].values
                
                
                if top_md.size == 0:
                    interp_func = interp1d(fourth_col, first_col_interpolated, bounds_error=False, fill_value='extrapolate')
                    top_md = interp_func(top_tvd)

                if bottom_md.size == 0:
                    interp_func = interp1d(fourth_col, first_col_interpolated, bounds_error=False, fill_value='extrapolate')
                    bottom_md = interp_func(bottom_tvd)

                copy_dict['Top_MD'][index] = round(float(top_md), 5)
                copy_dict['Bottom_MD'][index] = round(float(bottom_md), 5)

    return copy_dict
 

In [10]:
dict_final = process_well_data_3('C:/HV/Seismic/datas/test_3_5/Inklinometria', list_of_dicts)

In [11]:
df_table_coor = pd.DataFrame(dict_final)

In [12]:
df_table_coor

,wells,inlines,xlines,Lateral_trend,Top_TVD,Bottom_TVD,Top_MD,Bottom_MD
0,673R,449194.772480,7.008929e+06,1.0,-2783.899170,-2861.688721,2911.25042,2989.08781
1,676R,454672.154278,7.009559e+06,1.0,-2782.894775,-2858.207031,2922.06891,2997.47451
2,660R,450928.636400,7.011087e+06,0.0,-2786.688477,-2875.321045,2900.17848,2988.81104
3,661R,457638.951200,7.011792e+06,0.0,-2764.798584,-2843.564209,2893.78858,2972.55421
4,669R,451334.990000,7.011850e+06,3.0,-2780.111572,-2871.307129,2890.92157,2982.11713
5,663R,447671.940000,7.012181e+06,3.0,-2769.456787,-2860.164795,2872.91679,2963.62480
6,662R,455158.133924,7.014716e+06,2.0,-2762.990479,-2843.939941,2876.09048,2957.03994


In [107]:
def process_txt_files(folder_path, df):
    
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            
            name_part = filename.split('_')[0]
            
            
            well_row = df[df['wells'] == name_part]
            
            if not well_row.empty:
                top_u1 = well_row['Top_MD'].values[0]
                #bottom_u1 = well_row['Bottom_MD'].values[0] 
                bottom_u1 = well_row['Bottom_MD'].values[0] - (well_row['Bottom_MD'].values[0] - top_u1) / 2
                file_path = os.path.join(folder_path, filename)

                with open(file_path, 'r') as file:
                    
                    for line in file:
                        if line.startswith('~Ascii'):
                            break

                    
                    values = []
                    for line in file:
                        parts = line.split()
                        if len(parts) >= 2:
                            try:
                                first_value = float(parts[0]) 
                                
                                if first_value >= top_u1 and first_value <= bottom_u1:
                                    
                                    value = parts[1]
                                    if value != '-999.250000':
                                        values.append(float(value))
                            except ValueError:
                                continue  

                    
                    if values:
                        
                        value_counts = Counter(values)
                        
                        most_common_value, most_common_count = value_counts.most_common(1)[0]

                        entry = {
                            'wells': name_part,
                            'наиболее_частое_значение': most_common_value,
                            'количество': most_common_count
                        }
                         
                        for value, count in value_counts.items():
                            if value != most_common_value:
                                entry[f'значение_{value}'] = value
                                entry[f'количество_{value}'] = count

                        data.append(entry)

    result_df = pd.DataFrame(data)
    result_df.fillna(0, inplace=True)

    return result_df


In [108]:
folder_path = 'C:/HV/Seismic/datas/test_3_5/Log_facii'   # Укажите путь к вашей папке
df_facii  = process_txt_files(folder_path, df_table_coor)

In [109]:
df_facii

,wells,наиболее_частое_значение,количество,значение_1.0,количество_1.0,значение_0.0,количество_0.0
0,660R,0.0,334,1.0,109.0,0.0,0.0
1,661R,1.0,200,0.0,0.0,0.0,148.0
2,662R,1.0,144,0.0,0.0,0.0,134.0
3,663R,1.0,245,0.0,0.0,0.0,191.0
4,669R,0.0,373,1.0,83.0,0.0,0.0
5,673R,0.0,311,1.0,78.0,0.0,0.0
6,676R,0.0,229,1.0,148.0,0.0,0.0


In [104]:
def process_txt_files_2(folder_path, df):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            name_part = filename.split('_')[0]
            well_row = df[df['wells'] == name_part]
            
            if not well_row.empty:
                top_u1 = well_row['Top_MD'].values[0]
                #bottom_u1 = well_row['Bottom_MD'].values[0]
                bottom_u1 = well_row['Bottom_MD'].values[0] - (well_row['Bottom_MD'].values[0] - top_u1) / 2
                file_path = os.path.join(folder_path, filename)

                with open(file_path, 'r') as file:
                    for line in file:
                        if line.startswith('~Ascii'):
                            break

                    values = []
                    for line in file:
                        parts = line.split()
                        if len(parts) >= 2:
                            try:
                                first_value = float(parts[0])
                                
                                if top_u1 <= first_value <= bottom_u1:
                                    value = parts[1]
                                    if value != '-999.250000':
                                        values.append(float(value))
                            except ValueError:
                                continue  

                    # Подсчет всех значений, кроме -999.250000
                    value_counts = Counter(values)
                    
                    entry = {
                        'wells': name_part
                    }

                    # Добавление всех различных значений и их количества
                    for value, count in value_counts.items():
                        entry[f'значение_{value}'] = value
                        entry[f'количество_{value}'] = count

                    data.append(entry)

    # Создаем DataFrame из собранных данных
    result_df = pd.DataFrame(data)
    result_df.fillna(0, inplace=True)

    return result_df    
    


In [105]:
folder_path = 'C:/HV/Seismic/datas/test_3_5/Log_facii'   
df_facii_2  = process_txt_files_2(folder_path, df_table_coor)

In [106]:
df_facii_2

,wells,значение_0.0,количество_0.0,значение_1.0,количество_1.0
0,660R,0.0,334,1.0,109
1,661R,0.0,148,1.0,200
2,662R,0.0,134,1.0,144
3,663R,0.0,191,1.0,245
4,669R,0.0,373,1.0,83
5,673R,0.0,311,1.0,78
6,676R,0.0,229,1.0,148


In [130]:
def plot_grouped_histogram_3(df):
    fig = go.Figure()

    
    well_names = df['wells'].unique()
    num_wells = len(well_names)

    
    bar_width = 0.8 / num_wells  
    x_positions = np.arange(num_wells)  

    for index, row in df.iterrows():
        well_name = row['wells']
        values = row[1::2].values  
        counts = row[2::2].values  

        well_index = np.where(well_names == well_name)[0][0]

        for i in range(len(values)):
            color = 'green' if counts[i] == max(counts) else 'yellow'  
            fig.add_trace(go.Bar(
                x=[well_index + i * bar_width],  
                y=[counts[i]],  
                name=f'{well_name} - значение {i}',  
                text=[f'{values[i]}: {counts[i]}'],  
                textposition='auto',
                marker_color=color,
                width=bar_width  
            ))

    fig.update_layout(
        title='Гистограммы для всех скважин',
        xaxis_title='Скважины',
        yaxis_title='Количество',
        barmode='group',  
        legend_title='Значения',
        margin=dict(l=50, r=50, t=50, b=100),
        xaxis=dict(tickvals=x_positions, ticktext=well_names)  
    )

    fig.show()


In [131]:
plot_grouped_histogram_3(df_facii_2)

In [112]:
df_finish = pd.merge(df_table_coor[['wells', 'Lateral_trend']], df_facii[['wells', 'наиболее_частое_значение']], on='wells', how='inner')

In [113]:
df_finish

,wells,Lateral_trend,наиболее_частое_значение
0,673R,1.0,0.0
1,676R,1.0,0.0
2,660R,0.0,0.0
3,661R,0.0,1.0
4,669R,3.0,0.0
5,663R,3.0,1.0
6,662R,2.0,1.0


In [120]:
def plot_with_trendline(df, x_col, y_col, name_col):
    """
    Функция для построения графика с линией тренда и значением R^2,
    а также отображением всех имен при наведении на точки.

    :param df: DataFrame, содержащий данные ['wells', 'Lateral_trend', 'наиболее_частое_значение']
    :param x_col: str, имя колонки для оси X
    :param y_col: str, имя колонки для оси Y
    :param name_col: str, имя колонки с именами для отображения
    """

    df[name_col] = df[name_col].astype(str)
    df[x_col] = df[x_col].astype(float)
    df['hover_names'] = df.groupby([x_col, y_col])[name_col].transform(lambda x: ', '.join(x))
    x = df[x_col]
    y = df[y_col]
    hover_names = df['hover_names']

    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    trendline = polynomial(x)

    r_squared = 1 - (np.sum((y - trendline) ** 2) / np.sum((y - np.mean(y)) ** 2))
    r_sqrt = round(np.sqrt(r_squared), 3)

    slope = coefficients[0]
    intercept = coefficients[1]
    equation = f'y = {slope:.2f}x + {intercept:.2f}'


    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Скважины',
        text=hover_names,  
        hoverinfo='text'  
    ))
    fig.add_trace(go.Scatter(x=x, y=trendline, mode='lines', name='Линия тренда'))

    fig.add_annotation(
        x=0.5,
        y=0.9,
        text=f'|R| = {r_sqrt:.2f}<br>{equation}',
        showarrow=False,
        font=dict(size=16),
        xref="paper",
        yref="paper"
    )

    fig.update_layout(
        title='Скважины с линией тренда',
        xaxis_title=x_col,
        yaxis_title=y_col
    )
    fig.show()
    
    if r_sqrt < 0.7:
        print('Плохая корреляция преобладающих фаций в скважинах с картой трендов')


In [121]:
plot_with_trendline(df_finish, 'Lateral_trend', 'наиболее_частое_значение', 'wells')
# из данных что мне дали полчуается вот такая картина, это нормально. Саша так сказала

Плохая корреляция преобладающих фаций в скважинах с картой трендов


In [133]:

df_finish['Lateral_trend'] = df_finish['Lateral_trend'].astype(str)

count_df = df_finish.groupby(['наиболее_частое_значение', 'Lateral_trend']).size().reset_index(name='Количество скважин')

# Создание графика
fig = px.bar(count_df, 
             x='наиболее_частое_значение', 
             y='Количество скважин', 
             color='Lateral_trend',  
             barmode='group',  
             title='Количество скважин по классам и сейсм классам',
             labels={'наиболее_частое_значение': 'наиболее_частое_значение', 'Количество скважин': 'Количество скважин'},
             text='Количество скважин')




# Показать график
fig.show()


In [59]:
#для примера
df_1 = pd.read_excel('C:/HV/Seismic/datas/test_3_5/SFA_SO.xlsx')

In [60]:
df_1 

,Скважина,Сейсм класс,Скв класс1
0,9,5,1
1,16,4,1
2,8,5,3
3,3,5,2
4,2,5,1
5,17,4,1
6,7,5,2
7,5,5,2
8,14,4,1
9,15,4,1


In [53]:
plot_with_trendline(df_1, 'Скв класс1', 'Сейсм класс', 'Скважина')

Плохая корреляция преобладающих фаций в скважинах с картой трендов
